In [4]:
import torch

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import random
from scipy.interpolate import UnivariateSpline  
import pickle
from sklearn.decomposition import PCA
import time
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
from pathlib import Path

from model import MV_LSTM

In [5]:
base_path = Path("../dataset")

In [6]:
from numpy import array
from numpy import hstack

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(0,len(sequences),100):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if i!=0 and end_ix > len(sequences):
            break
        
        sequences[i:end_ix,0]=np.insert(np.diff(sequences[i:end_ix,0]),0,0)
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix-33], sequences[end_ix-33:end_ix]
        
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [33]:
df = pd.read_csv(base_path/"modified_usa.csv")

is_nd = (df["State"] == "North Dakota")

df2 = df.copy(deep = True)

is_ken = (df["State"] == "Kentucky")

In [34]:
states = df["State"].unique()
states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [35]:
data = df[df.State.isin(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'])][["Confirmed", "Deaths", "lat", "long"]]


In [36]:
data2=df2[(is_ken)][['Confirmed','lat','long','Deaths']]
date=df2[(is_ken)][['Date','Confirmed']]

date["Date"] = pd.to_datetime(date["Date"], format = '%Y%m%d', errors = 'ignore')
date.set_index('Date', inplace = True)

In [37]:
n_features = 4 # this is number of parallel inputs
n_timesteps = 100 # this is number of timesteps

In [38]:
data.isna().sum(axis = 0)

Confirmed    0
Deaths       0
lat          0
long         0
dtype: int64

In [39]:
X, Y = split_sequences(data.values, n_timesteps)
print (X.shape,Y.shape) 

(9957, 67, 4) (9957, 33, 4)


In [40]:
alld=np.concatenate((X,Y),1)
alld=alld.reshape(alld.shape[0]*alld.shape[1],alld.shape[2])

scaler = MinMaxScaler()
scaler.fit(alld)
X=[scaler.transform(x) for x in X]
y=[scaler.transform(y) for y in Y]

X=np.array(X)
y=np.array(y)[:,:,0]

In [46]:
mv_net = MV_LSTM(n_features,67).cuda()
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-3)

train_episodes = 1

batch_size = 16
losses = []

mv_net.train()

for t in range(train_episodes):
    
    for b in range(0,len(X),batch_size):
       
        p = np.random.permutation(len(X))
        
        inpt = X[p][b:b+batch_size,:,:]
        target = y[p][b:b+batch_size,:]    
        
        x_batch = torch.tensor(inpt,dtype=torch.float32).cuda()    
        y_batch = torch.tensor(target,dtype=torch.float32).cuda()
       
        mv_net.init_hidden(x_batch.size(0))
        
        output = mv_net(x_batch) 
        
        
        all_batch=torch.cat((x_batch[:,:,0], y_batch), 1)
        
        
        loss = 1000*criterion(output.view(-1), all_batch.view(-1))  

        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 
    losses.append(loss.item())    
    print('step : ' , t , 'loss : ' , loss.item())

with open('../model/losses.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(losses, f,protocol=2)

step :  0 loss :  7.552993338322267e-05


In [42]:
data2x=data2
truth = data2

data2x.values[0:len(data2x),0]=np.insert(np.diff(data2x.values[0:len(data2x),0]),0,0)
data2x=scaler.transform(data2x) 


X_test = np.expand_dims(data2x, axis=0)
print (X_test.shape)
mv_net.init_hidden(1)


lstm_out = mv_net(torch.tensor(X_test[:,-67:,:],dtype=torch.float32).cuda())
lstm_out=lstm_out.reshape(1,100,1).cpu().data.numpy()

print (data2x[-67:,0],lstm_out)
actual_predictions = scaler.inverse_transform(np.tile(lstm_out, (1, 1,5))[0])[:,0]

print (data2.values[-67:,0],actual_predictions)

#actual_predictions=lstm_out


x = np.arange(0, 54, 1)
x2 = np.arange(0, 67, 1)
x3 = np.arange(0, 100, 10)
x4 = np.arange(0, 50, 1)


#save prediction
with open('./lstmdata/predict_indo8.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(pd.Series(actual_predictions), f,protocol=2)

(1, 37820, 4)
[0.96509094 0.96509346 0.96509597 0.96511862 0.96512617 0.9651312
 0.96513372 0.96513623 0.96514378 0.96515636 0.96516895 0.96518153
 0.96519159 0.96519914 0.96520418 0.96521172 0.96522179 0.96522682
 0.9652394  0.96524695 0.96525702 0.96525702 0.96526457 0.96527715
 0.9652847  0.96530483 0.96530986 0.96532244 0.96532748 0.96532748
 0.96534257 0.96534761 0.96536019 0.96537025 0.96538787 0.96539038
 0.96538787 0.96540297 0.9654231  0.96545581 0.96548097 0.9655011
 0.96551872 0.96553381 0.96555898 0.96558917 0.96560679 0.96562692
 0.96563195 0.96567473 0.96567976 0.96570241 0.96574015 0.96576783
 0.96580055 0.96580306 0.96582571 0.9658559  0.9659012  0.96598927
 0.96603708 0.96608237 0.96610251 0.9661176  0.96614277 0.96618051
 0.96618051] [[[0.9681258 ]
  [0.9680949 ]
  [0.9681351 ]
  [0.9681682 ]
  [0.96804404]
  [0.9681548 ]
  [0.9681589 ]
  [0.9681655 ]
  [0.9680719 ]
  [0.9679606 ]
  [0.9678945 ]
  [0.9679776 ]
  [0.96799433]
  [0.9681494 ]
  [0.96795684]
  [0.9678881 

ValueError: operands could not be broadcast together with shapes (100,5) (4,) (100,5) 